In [145]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots as sp
import plotly.figure_factory as ff
import plotly.express as px


In [146]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_test["SalePrice"]=-1

df=pd.concat([df_train,df_test])




In [147]:
from plotly.subplots import make_subplots

train_columns=[col for col in list(df.columns) if col not in ['Id','SalePrice']]
target_column='SalePrice'

df_test[train_columns].describe()

df[train_columns]

cat_columns=[col for col in train_columns if df[col].dtype=='object']

for col in cat_columns:
    df[col]=df[col].astype('category')  


def plot_hist_for_all_features(df:pd.DataFrame,col_name:list[str],target_column:str):

    for col in col_name:
        fig=px.scatter(df,x=df[col],y=df[target_column],title=col)
        fig.show()
        




In [149]:
from sklearn.preprocessing import LabelEncoder,StandardScaler

numerical_columns=[col for col in train_columns if col not in cat_columns]

#convert categorical variables to numerical
encoders={}
scalers={}

for col in cat_columns:

    encoder=LabelEncoder()
    scaler=StandardScaler()
    df[col]=encoder.fit_transform(df[col])
    df[col]=scaler.fit_transform(df[[col]])
    encoders[col]=encoder
    scalers[col]=scaler

for col in numerical_columns:
    scaler=StandardScaler()
    df[col]=scaler.fit_transform(df[[col]])
    scalers[col]=scaler

X_train=df.loc[df[target_column]!=-1,train_columns]
y_train=df.loc[df[target_column]!=-1,target_column]

target_scaler=StandardScaler()
y_train=target_scaler.fit_transform(pd.DataFrame(y_train))


In [150]:
""## Let's remove from the dataset the 3% of outliers
from sklearn.ensemble import IsolationForest

fraction_outliers=0.005
isolation_model=IsolationForest(contamination=fraction_outliers)

isolation_model.fit(X_train)

outliers_mask=isolation_model.predict(X_train)

X_train=X_train[outliers_mask==1]
y_train=y_train[outliers_mask==1]


""

''

In [151]:
plot_hist_for_all_features(df_train,train_columns[:5],target_column)

In [152]:
import optuna
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score


def xgb_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'seed': 42
    }

    model = XGBRegressor(**params)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    return -cv_scores.mean()


def lgbm_objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "verbosity": -1,
        "silent": True
    }

    model = LGBMRegressor(**params)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    return -cv_scores.mean()





In [153]:
X_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.067331,-0.046038,-0.184481,-0.217879,0.064249,0.257283,0.746669,0.316671,-0.030863,0.588555,-0.21626,-1.248476,-0.046258,-0.009816,-0.419174,1.031721,0.646183,-0.507284,1.046258,0.896833,-0.482926,-0.116923,0.742177,0.749790,-1.389009,0.523129,-0.755793,0.376461,0.834618,-0.312279,0.234254,0.584281,-0.454809,0.580807,0.281997,-0.293080,-0.934702,-0.444252,-0.103206,-0.880324,0.26829,0.300211,-0.773861,1.207379,-0.101197,0.413547,1.086650,-0.249810,0.781366,1.232599,0.169927,-0.207698,-0.416171,0.986849,0.255925,-0.924311,0.839596,-0.767783,0.973286,-0.316551,0.306475,0.348840,0.137641,0.071758,0.31503,-0.740760,0.200006,-0.359601,-0.103331,-0.285935,-0.06315,0.053507,0.464024,0.188401,-0.089592,-1.552184,0.157646,0.319053,0.204967
1,-0.873616,-0.046038,0.458190,-0.072044,0.064249,0.257283,0.746669,0.316671,-0.030863,-0.658174,-0.21626,1.941064,-1.190557,-0.009816,-0.419174,-0.536817,-0.063185,2.188279,0.154764,-0.395604,-0.482926,-0.116923,-0.507918,-0.658056,0.769946,-0.569991,0.668455,0.376461,-0.541017,-0.312279,0.234254,-1.153382,-1.528910,1.177910,0.281997,-0.293080,-0.629788,0.477029,-0.103206,-0.880324,0.26829,0.300211,0.261075,-0.785025,-0.101197,-0.471891,-0.819398,3.821109,0.781366,-0.756321,0.169927,-0.207698,0.781859,-0.287758,0.255925,0.623632,0.124457,-0.767783,-0.082653,-0.316551,0.306475,-0.059782,0.137641,0.071758,0.31503,1.614879,-0.702843,-0.359601,-0.103331,-0.285935,-0.06315,0.053507,0.464024,0.188401,-0.089592,-0.446925,-0.602962,0.319053,0.204967
2,0.067331,-0.046038,-0.055946,0.137197,0.064249,0.257283,-1.381776,0.316671,-0.030863,0.588555,-0.21626,-1.248476,-0.046258,-0.009816,-0.419174,1.031721,0.646183,-0.507284,0.980221,0.848965,-0.482926,-0.116923,0.742177,0.749790,-1.389009,0.333506,-0.755793,0.376461,0.834618,-0.312279,0.234254,-0.284550,-0.454809,0.097856,0.281997,-0.293080,-0.288467,-0.299025,-0.103206,-0.880324,0.26829,0.300211,-0.610718,1.235375,-0.101197,0.563755,1.086650,-0.249810,0.781366,1.232599,0.169927,-0.207698,-0.416171,-0.287758,0.255925,0.623632,0.124457,-0.767783,0.895069,-0.316551,0.306475,0.627446,0.137641,0.071758,0.31503,-0.740760,-0.081209,-0.359601,-0.103331,-0.285935,-0.06315,0.053507,0.464024,0.188401,-0.089592,1.026753,0.157646,0.319053,0.204967
3,0.302568,-0.046038,-0.398704,-0.078385,0.064249,0.257283,-1.381776,0.316671,-0.030863,-1.904903,-0.21626,-1.080605,-0.046258,-0.009816,-0.419174,1.031721,0.646183,-0.507284,-1.859351,-0.682812,-0.482926,-0.116923,1.054701,1.312929,0.769946,-0.569991,0.668455,0.376461,-1.916652,0.771599,-2.620806,0.584281,-1.528910,-0.494856,0.281997,-0.293080,-0.047266,-0.671168,-0.103206,0.267617,0.26829,0.300211,-0.506205,0.978742,-0.101197,0.427382,1.086650,-0.249810,-1.027363,-0.756321,0.169927,-0.207698,-0.416171,0.349546,0.255925,0.623632,-1.305821,1.302093,0.777742,0.798060,1.619684,0.785323,0.137641,0.071758,0.31503,-0.740760,-0.184815,3.874967,-0.103331,-0.285935,-0.06315,0.053507,0.464024,0.188401,-0.089592,-1.552184,-1.363569,0.319053,-3.505415
4,0.067331,-0.046038,0.629569,0.518903,0.064249,0.257283,-1.381776,0.316671,-0.030863,-0.658174,-0.21626,0.430229,-0.046258,-0.009816,-0.419174,1.031721,1.355551,

In [154]:
# Creazione degli studi Optuna
study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(xgb_objective, n_trials=50)

[I 2025-04-01 22:47:25,490] A new study created in memory with name: no-name-aacaf009-0b3a-42e5-a286-d1acb5e228c2
[I 2025-04-01 22:47:26,636] Trial 0 finished with value: 0.1809583362864599 and parameters: {'n_estimators': 470, 'max_depth': 5, 'learning_rate': 0.21180828470598295, 'min_child_weight': 2, 'subsample': 0.8230689398065243, 'colsample_bytree': 0.5322323225134027, 'gamma': 6.354882742613521, 'reg_alpha': 6.821523967614976, 'reg_lambda': 7.763977731933234}. Best is trial 0 with value: 0.1809583362864599.
[I 2025-04-01 22:47:27,244] Trial 1 finished with value: 0.19067842436977647 and parameters: {'n_estimators': 182, 'max_depth': 4, 'learning_rate': 0.06581458090946252, 'min_child_weight': 3, 'subsample': 0.8010133160186571, 'colsample_bytree': 0.5123274875931199, 'gamma': 9.279744455727018, 'reg_alpha': 6.48294441399532, 'reg_lambda': 9.9029686161403}. Best is trial 0 with value: 0.1809583362864599.
[I 2025-04-01 22:47:29,150] Trial 2 finished with value: 0.1855868247417896 

In [155]:
study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(lgbm_objective, n_trials=50)

[I 2025-04-01 22:48:15,472] A new study created in memory with name: no-name-ef7e3d97-0610-4105-9477-dd859300bc67
[I 2025-04-01 22:48:18,853] Trial 0 finished with value: 0.13554082667198025 and parameters: {'n_estimators': 295, 'learning_rate': 0.16459427330299722, 'num_leaves': 1520, 'max_depth': 5}. Best is trial 0 with value: 0.13554082667198025.
[I 2025-04-01 22:48:22,891] Trial 1 finished with value: 0.1271290880450533 and parameters: {'n_estimators': 434, 'learning_rate': 0.09637741176365949, 'num_leaves': 2080, 'max_depth': 4}. Best is trial 1 with value: 0.1271290880450533.
[I 2025-04-01 22:48:29,085] Trial 2 finished with value: 0.1405317166007958 and parameters: {'n_estimators': 499, 'learning_rate': 0.08035213546775877, 'num_leaves': 360, 'max_depth': 7}. Best is trial 1 with value: 0.1271290880450533.
[I 2025-04-01 22:48:32,437] Trial 3 finished with value: 0.1360214254839313 and parameters: {'n_estimators': 252, 'learning_rate': 0.12805014685161892, 'num_leaves': 1460, 'm

In [156]:
# Stampa dei migliori parametri per ogni modello
print("Migliori parametri XGBoost:", study_xgb.best_params)
print("Miglior valore XGBoost:", study_xgb.best_value)

print("Migliori parametri LightGBM:", study_lgbm.best_params)
print("Miglior valore LightGBM:", study_lgbm.best_value)



Migliori parametri XGBoost: {'n_estimators': 159, 'max_depth': 6, 'learning_rate': 0.07405923289435414, 'min_child_weight': 8, 'subsample': 0.6330660608466465, 'colsample_bytree': 0.9130130244378137, 'gamma': 0.0855901417123143, 'reg_alpha': 1.0571732605196185, 'reg_lambda': 3.93058137060822}
Miglior valore XGBoost: 0.11811112182679291
Migliori parametri LightGBM: {'n_estimators': 157, 'learning_rate': 0.03968366238918349, 'num_leaves': 2560, 'max_depth': 11}
Miglior valore LightGBM: 0.1203296335548882


In [157]:
## Lets train the models

xgb_model=XGBRegressor(**study_xgb.best_params)
lgbm_model=LGBMRegressor(**study_lgbm.best_params)


xgb_model.fit(X_train,y_train)
lgbm_model.fit(X_train,y_train)




LGBMRegressor(learning_rate=0.03968366238918349, max_depth=11, n_estimators=157,
              num_leaves=2560)

In [158]:
from sklearn.ensemble import StackingRegressor

estimators=[('xgb',xgb_model),('lgbm',lgbm_model)]

stacking_model=StackingRegressor(estimators=estimators,final_estimator=LinearRegression())

stacking_model.fit(X_train,y_train)

/Users/carlo/Library/Caches/pypoetry/virtualenvs/3-_house_price_predictor-gE_A7rtz-py3.12/lib/python3.12/site-packages/sklearn/ensemble/_stacking.py:1060: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



StackingRegressor(estimators=[('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.9130130244378137,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            feature_weights=None,
                                            gamma=0.0855901417123143,
                                            grow_policy=None,
                                            importance_t...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None, min_child_weight=8,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=159, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                              ('lgbm',
                               LGBMRegressor(learning_rate=0.03968366238918349,
                                             max_depth=11, n_estimators=157,
                                             num_leaves=2560))],
                  final_estimator=LinearRegression())

In [159]:
X_test=df.loc[df[target_column]==-1,train_columns]

In [163]:
X_test

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,-0.873616,RH,0.458190,0.184371,1,2,3,3,0,4,0,12,1,2,0,2,-0.772552,0.391237,-0.340510,-1.113625,1,1,12,13,3,-0.569991,3,4,1,3,3,3,4,0.058342,3,0.558102,-0.661645,-0.385253,1,4,1,4,-0.671897,-0.785025,-0.101197,-1.195262,-0.819398,-0.249810,-1.027363,-0.756321,-1.045801,-0.207698,3,-0.925062,6,-0.924311,5,1,-0.669286,2,-1.006733,1.193945,4,4,2,0.365916,-0.702843,-0.359601,-0.103331,1.850256,-0.06315,3,2,4,-0.089592,-0.078505,1.678860,8,4
1,-0.873616,RL,0.501035,0.519791,1,2,0,3,0,0,0,12,2,2,0,2,-0.063185,0.391237,-0.439565,-1.257229,3,1,13,14,1,0.032340,3,4,1,3,3,3,0,1.057173,5,-0.293080,-0.352180,0.629063,1,4,1,4,0.431865,-0.785025,-0.101197,-0.339470,-0.819398,-0.249810,-1.027363,1.232599,0.169927,-0.207698,2,-0.287758,6,-0.924311,5,1,-0.786612,2,-1.006733,-0.747010,4,4,2,2.365838,-0.170014,-0.359601,-0.103331,-0.285935,-0.06315,3,4,0,21.944410,-0.078505,1.678860,8,4
2,0.067331,RL,0.201122,0.464374,1,2,0,3,0,4,0,8,2,2,0,5,-0.772552,-0.507284,0.848148,0.657493,1,1,12,13,3,-0.569991,3,4,2,2,3,3,2,0.767402,5,-0.293080,-0.964284,-0.280872,1,2,1,4,-0.590326,0.850426,-0.101197,0.253457,-0.819398,-0.249810,0.781366,1.232599,0.169927,-0.207698,3,-0.287758,6,0.623632,4,1,0.738633,0,0.306475,0.042373,4,4,2,0.935064,-0.199616,-0.359601,-0.103331,-0.285935,-0.06315,3,2,4,-0.089592,-1.183764,1.678860,8,4
3,0.067331,RL,0.372501,-0.024109,1,2,0,3,0,4,0,8,2,2,0,5,-0.063185,0.391237,0.881166,0.657493,1,1,12,13,1,-0.458448,3,4,2,3,3,3,2,0.352503,5,-0.293080,-0.538769,-0.285410,1,0,1,4,-0.595424,0.796766,-0.101197,0.204046,-0.819398,-0.249810,0.781366,1.232599,0.169927,-0.207698,2,0.349546,6,0.623632,2,1,0.777742,0,0.306475,-0.013348,4,4,2,2.104979,-0.170014,-0.359601,-0.103331,-0.285935,-0.06315,3,4,4,-0.089592,-0.078505,1.678860,8,4
4,1.478753,RL,-1.127064,-0.654748,1,2,0,1,0,4,0,22,2,2,4,2,1.355551,-0.507284,0.683057,0.370284,1,1,6,6,3,-0.569991,2,4,2,2,3,3,0,-0.391680,5,-0.293080,1.038136,0.517874,1,0,1,4,0.306959,-0.785025,-0.101197,-0.436315,-0.819398,-0.249810,0.781366,-0.756321,-1.045801,-0.207698,2,-0.925062,6,-0.924311,5,1,0.543089,1,0.306475,0.153816,4,4,2,-0.740760,0.510823,-0.359601,-0.103331,2.277494,-0.06315,3,4,4,-0.089592,-1.920603,1.678860,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2.419700,RM,-2.069648,-1.043937,1,2,3,3,0,4,0,10,2,2,3,5,-1.481920,1.289758,-0.043346,-0.682812,1,1,5,5,3,-0.569991,3,4,1,3,3,3,5,-0.969026,5,-0.293080,-0.033614,-1.147693,1,2,1,4,-1.564083,0.488807,-0.101197,-0.807883,-0.819398,-0.249810,-1.027363,1.232599,0.169927,-0.207698,3,-0.925062,6,-0.924311,5,6,NaN,3,-2.319941,-2.195761,5,5,2,-0.740760,-0.702843,-0.359601,-0.103331,-0.285935,-0.06315,3,4,4,-0.089592,-0.078505,-1.363569,8,4
1455,2.419700,RM,-2.069648,-1.049263,1,2,3,3,0,4,0,10,2,2,4,5,-1.481920,-0.507284,-0.043346,-0.682812,1,1,5,5,3,-0.569991,3,4,1,3,3,3,4,-0.415828,5,-0.293080,-0.607034,-1.147693,1,4,1,4,-1.564083,0.488807,-0.10119

In [166]:
X_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.067331,-0.046038,-0.184481,-0.217879,0.064249,0.257283,0.746669,0.316671,-0.030863,0.588555,-0.21626,-1.248476,-0.046258,-0.009816,-0.419174,1.031721,0.646183,-0.507284,1.046258,0.896833,-0.482926,-0.116923,0.742177,0.749790,-1.389009,0.523129,-0.755793,0.376461,0.834618,-0.312279,0.234254,0.584281,-0.454809,0.580807,0.281997,-0.293080,-0.934702,-0.444252,-0.103206,-0.880324,0.26829,0.300211,-0.773861,1.207379,-0.101197,0.413547,1.086650,-0.249810,0.781366,1.232599,0.169927,-0.207698,-0.416171,0.986849,0.255925,-0.924311,0.839596,-0.767783,0.973286,-0.316551,0.306475,0.348840,0.137641,0.071758,0.31503,-0.740760,0.200006,-0.359601,-0.103331,-0.285935,-0.06315,0.053507,0.464024,0.188401,-0.089592,-1.552184,0.157646,0.319053,0.204967
1,-0.873616,-0.046038,0.458190,-0.072044,0.064249,0.257283,0.746669,0.316671,-0.030863,-0.658174,-0.21626,1.941064,-1.190557,-0.009816,-0.419174,-0.536817,-0.063185,2.188279,0.154764,-0.395604,-0.482926,-0.116923,-0.507918,-0.658056,0.769946,-0.569991,0.668455,0.376461,-0.541017,-0.312279,0.234254,-1.153382,-1.528910,1.177910,0.281997,-0.293080,-0.629788,0.477029,-0.103206,-0.880324,0.26829,0.300211,0.261075,-0.785025,-0.101197,-0.471891,-0.819398,3.821109,0.781366,-0.756321,0.169927,-0.207698,0.781859,-0.287758,0.255925,0.623632,0.124457,-0.767783,-0.082653,-0.316551,0.306475,-0.059782,0.137641,0.071758,0.31503,1.614879,-0.702843,-0.359601,-0.103331,-0.285935,-0.06315,0.053507,0.464024,0.188401,-0.089592,-0.446925,-0.602962,0.319053,0.204967
2,0.067331,-0.046038,-0.055946,0.137197,0.064249,0.257283,-1.381776,0.316671,-0.030863,0.588555,-0.21626,-1.248476,-0.046258,-0.009816,-0.419174,1.031721,0.646183,-0.507284,0.980221,0.848965,-0.482926,-0.116923,0.742177,0.749790,-1.389009,0.333506,-0.755793,0.376461,0.834618,-0.312279,0.234254,-0.284550,-0.454809,0.097856,0.281997,-0.293080,-0.288467,-0.299025,-0.103206,-0.880324,0.26829,0.300211,-0.610718,1.235375,-0.101197,0.563755,1.086650,-0.249810,0.781366,1.232599,0.169927,-0.207698,-0.416171,-0.287758,0.255925,0.623632,0.124457,-0.767783,0.895069,-0.316551,0.306475,0.627446,0.137641,0.071758,0.31503,-0.740760,-0.081209,-0.359601,-0.103331,-0.285935,-0.06315,0.053507,0.464024,0.188401,-0.089592,1.026753,0.157646,0.319053,0.204967
3,0.302568,-0.046038,-0.398704,-0.078385,0.064249,0.257283,-1.381776,0.316671,-0.030863,-1.904903,-0.21626,-1.080605,-0.046258,-0.009816,-0.419174,1.031721,0.646183,-0.507284,-1.859351,-0.682812,-0.482926,-0.116923,1.054701,1.312929,0.769946,-0.569991,0.668455,0.376461,-1.916652,0.771599,-2.620806,0.584281,-1.528910,-0.494856,0.281997,-0.293080,-0.047266,-0.671168,-0.103206,0.267617,0.26829,0.300211,-0.506205,0.978742,-0.101197,0.427382,1.086650,-0.249810,-1.027363,-0.756321,0.169927,-0.207698,-0.416171,0.349546,0.255925,0.623632,-1.305821,1.302093,0.777742,0.798060,1.619684,0.785323,0.137641,0.071758,0.31503,-0.740760,-0.184815,3.874967,-0.103331,-0.285935,-0.06315,0.053507,0.464024,0.188401,-0.089592,-1.552184,-1.363569,0.319053,-3.505415
4,0.067331,-0.046038,0.629569,0.518903,0.064249,0.257283,-1.381776,0.316671,-0.030863,-0.658174,-0.21626,0.430229,-0.046258,-0.009816,-0.419174,1.031721,1.355551,

In [167]:
X_test=df.loc[df[target_column]==-1,train_columns]






In [168]:
test_index=pd.read_csv('test.csv')['Id']

In [169]:
X_test


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,-0.873616,-1.555991,0.458190,0.184371,0.064249,0.257283,0.746669,0.316671,-0.030863,0.588555,-0.21626,-0.073382,-1.190557,-0.009816,-0.419174,-0.536817,-0.772552,0.391237,-0.340510,-1.113625,-0.482926,-0.116923,0.742177,0.749790,0.769946,-0.569991,0.668455,0.376461,-0.541017,0.771599,0.234254,0.584281,0.619291,0.058342,-1.694351,0.558102,-0.661645,-0.385253,-0.103206,1.415558,0.26829,0.300211,-0.671897,-0.785025,-0.101197,-1.195262,-0.819398,-0.249810,-1.027363,-0.756321,-1.045801,-0.207698,0.781859,-0.925062,0.255925,-0.924311,0.839596,-0.767783,-0.669286,0.798060,-1.006733,1.193945,0.137641,0.071758,0.31503,0.365916,-0.702843,-0.359601,-0.103331,1.850256,-0.06315,0.053507,-1.368839,0.188401,-0.089592,-0.078505,1.678860,0.319053,0.204967
1,-0.873616,-0.046038,0.501035,0.519791,0.064249,0.257283,-1.381776,0.316671,-0.030863,-1.904903,-0.21626,-0.073382,-0.046258,-0.009816,-0.419174,-0.536817,-0.063185,0.391237,-0.439565,-1.257229,1.953824,-0.116923,1.054701,1.031360,-1.389009,0.032340,0.668455,0.376461,-0.541017,0.771599,0.234254,0.584281,-1.528910,1.057173,0.281997,-0.293080,-0.352180,0.629063,-0.103206,1.415558,0.26829,0.300211,0.431865,-0.785025,-0.101197,-0.339470,-0.819398,-0.249810,-1.027363,1.232599,0.169927,-0.207698,-0.416171,-0.287758,0.255925,-0.924311,0.839596,-0.767783,-0.786612,0.798060,-1.006733,-0.747010,0.137641,0.071758,0.31503,2.365838,-0.170014,-0.359601,-0.103331,-0.285935,-0.06315,0.053507,0.464024,-9.676048,21.944410,-0.078505,1.678860,0.319053,0.204967
2,0.067331,-0.046038,0.201122,0.464374,0.064249,0.257283,-1.381776,0.316671,-0.030863,0.588555,-0.21626,-0.744864,-0.046258,-0.009816,-0.419174,1.031721,-0.772552,-0.507284,0.848148,0.657493,-0.482926,-0.116923,0.742177,0.749790,0.769946,-0.569991,0.668455,0.376461,0.834618,-0.312279,0.234254,0.584281,-0.454809,0.767402,0.281997,-0.293080,-0.964284,-0.280872,-0.103206,0.267617,0.26829,0.300211,-0.590326,0.850426,-0.101197,0.253457,-0.819398,-0.249810,0.781366,1.232599,0.169927,-0.207698,0.781859,-0.287758,0.255925,0.623632,0.124457,-0.767783,0.738633,-1.431163,0.306475,0.042373,0.137641,0.071758,0.31503,0.935064,-0.199616,-0.359601,-0.103331,-0.285935,-0.06315,0.053507,-1.368839,0.188401,-0.089592,-1.183764,1.678860,0.319053,0.204967
3,0.067331,-0.046038,0.372501,-0.024109,0.064249,0.257283,-1.381776,0.316671,-0.030863,0.588555,-0.21626,-0.744864,-0.046258,-0.009816,-0.419174,1.031721,-0.063185,0.391237,0.881166,0.657493,-0.482926,-0.116923,0.742177,0.749790,-1.389009,-0.458448,0.668455,0.376461,0.834618,0.771599,0.234254,0.584281,-0.454809,0.352503,0.281997,-0.293080,-0.538769,-0.285410,-0.103206,-0.880324,0.26829,0.300211,-0.595424,0.796766,-0.101197,0.204046,-0.819398,-0.249810,0.781366,1.232599,0.169927,-0.207698,-0.416171,0.349546,0.255925,0.623632,-1.305821,-0.767783,0.777742,-1.431163,0.306475,-0.013348,0.137641,0.071758,0.31503,2.104979,-0.170014,-0.359601,-0.103331,-0.285935,-0.06315,0.053507,0.464024,0.188401,-0.089592,-0.078505,1.678860,0.319053,0.204967
4,1.478753,-0.046038,-1.127064,-0.654748,0.064249,0.257283,-1.381776,-2.523149,-0.030863,0.588555,-0.21626,1.605323,-0.046258,-0.009816,2.896733,-0.536817,1.355551,

In [170]:
predictions=xgb_model.predict(X_test)

predictions=target_scaler.inverse_transform(pd.DataFrame(predictions))

pd.DataFrame(predictions,index=test_index,columns=['SalePrice']).to_csv('submission7.csv')